## Optimal electricity price plan 

In [246]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [247]:
df = pd.read_hdf('data/h5/cleaned_data.h5')
df = df[df['OBT_PAVADINIMAS'] == 'Butas']
df['PL_T'] = pd.to_datetime(df['PL_T'])

df

,TINKLAS,OBT_PAVADINIMAS,OBJ_GV_TIPAS,OBJ_NUMERIS,P+,PL_T,P-
2,Alytaus regiono tinklas,Butas,Ne GV,37502,0.0500,2020-06-30 00:00:00,0.0
5,Vilniaus regiono tinklas,Butas,Ne GV,600022,0.0230,2020-06-30 00:00:00,0.0
17,Kauno regiono tinklas,Butas,N,197974,0.1058,2020-06-30 00:00:00,0.0
23,Vilniaus regiono tinklas,Butas,Ne GV,535458,0.0400,2020-06-30 00:00:00,0.0
43,Kauno regiono tinklas,Butas,Ne GV,193160,0.1340,2020-06-30 00:00:00,0.0
...,...,...,...,...,...,...,...
11816427,Vilniaus regiono tinklas,Butas,Ne GV,578418,0.1070,2022-05-01 01:00:00,0.0
11816434,Vilniaus regiono tinklas,Butas,Ne GV,585738,0.0260,2022-05-01 01:00:00,0.0
11816450,Vilniaus regiono tinklas,Butas,Ne GV,578178,0.0940,2022-05-01 01:00:00,0.0
11816458,Vilniaus regiono tinklas,Butas,Ne GV,575298,0.0380,2022-05-01 01:00:00,0.0


In [248]:
remaining_objects = df['OBJ_NUMERIS'].nunique()

print(f"Number of apartments: {remaining_objects}")

Number of apartments: 71


In [249]:
df_nordpool = pd.read_hdf('data/h5/electricity_prices_2020-2022.h5')
df_nordpool['datetime'] = pd.to_datetime(df_nordpool['datetime'])
df_nordpool

C:\Users\baubl\AppData\Local\Temp\ipykernel_39008\3956794645.py:2: UserWarning:

Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.



,datetime,price
0,2022-05-31 23:00:00,240.01
1,2022-05-31 22:00:00,247.51
2,2022-05-31 21:00:00,258.67
3,2022-05-31 20:00:00,255.02
4,2022-05-31 19:00:00,234.97
...,...,...
17515,2020-06-01 04:00:00,4.56
17516,2020-06-01 03:00:00,4.54
17517,2020-06-01 02:00:00,4.16
17518,2020-06-01 01:00:00,4.54


In [250]:
df['month'] = df['PL_T'].dt.to_period('M')

# Group by apartment and month, sum up electricity consumption
monthly_consumption = df.groupby(['OBJ_NUMERIS', 'month'])['P+'].sum().reset_index()
monthly_consumption

,OBJ_NUMERIS,month,P+
0,37082,2020-06,227.773
1,37082,2020-07,257.797
2,37082,2020-08,259.931
3,37082,2020-09,297.619
4,37082,2020-10,340.508
...,...,...,...
1699,802602,2022-01,270.754
1700,802602,2022-02,248.854
1701,802602,2022-03,261.863
1702,802602,2022-04,264.816


In [251]:
# Merge consumtion and Nordpool DataFrames
df_merged = df.merge(df_nordpool, how='inner', left_on='PL_T', right_on='datetime')
df_merged

,TINKLAS,OBT_PAVADINIMAS,OBJ_GV_TIPAS,OBJ_NUMERIS,P+,PL_T,P-,month,datetime,price
0,Alytaus regiono tinklas,Butas,Ne GV,37502,0.0500,2020-06-30 00:00:00,0.0,2020-06,2020-06-30 00:00:00,22.82
1,Vilniaus regiono tinklas,Butas,Ne GV,600022,0.0230,2020-06-30 00:00:00,0.0,2020-06,2020-06-30 00:00:00,22.82
2,Kauno regiono tinklas,Butas,N,197974,0.1058,2020-06-30 00:00:00,0.0,2020-06,2020-06-30 00:00:00,22.82
3,Vilniaus regiono tinklas,Butas,Ne GV,535458,0.0400,2020-06-30 00:00:00,0.0,2020-06,2020-06-30 00:00:00,22.82
4,Kauno regiono tinklas,Butas,Ne GV,193160,0.1340,2020-06-30 00:00:00,0.0,2020-06,2020-06-30 00:00:00,22.82
...,...,...,...,...,...,...,...,...,...,...
1243915,Vilniaus regiono tinklas,Butas,Ne GV,578418,0.1070,2022-05-01 01:00:00,0.0,2022-05,2022-05-01 01:00:00,130.57
1243916,Vilniaus regiono tinklas,Butas,Ne GV,585738,0.0260,2022-05-01 01:00:00,0.0,2022-05,2022-05-01 01:00:00,130.57
1243917,Vilniaus regiono tinklas,Butas,Ne GV,578178,0.0940,2022-05-01 01:00:00,0.0,2022-05,2022-05-01 01:00:00,130.57
1243918,Vilniaus regiono tinklas,Butas,Ne GV,575298,0.0380,2022-05-01 01:00:00,0.0,2022-05,2022-05-01 01:00:00,130.57


##### Plan (Minimalus) - fixed fee + 0.22 EUR/kWh

In [252]:
df_minimalus = df_merged.groupby(['month', 'OBJ_NUMERIS'])['P+'].sum().reset_index()

# Minimalus plan cost calculation
df_minimalus['Minimalus'] = 1.00 + 0.22 * df_minimalus['P+']

##### Plan (Standartinis) - higher fixed fee + 0.20 EUR/kWh

In [253]:
df_standartinis = df_merged.groupby(['month', 'OBJ_NUMERIS'])['P+'].sum().reset_index()

# Standartinis plan cost calculation
df_standartinis['Standartinis'] = 2.00 + 0.20 * df_standartinis['P+']

##### Plan (Lankstus) - fixed fee + 0.08 EUR/kWh + NordPool hourly price

In [254]:
# Calculate hourly costs
df_merged['Lankstus_Hourly_Cost'] = df_merged['P+'] * (0.08 + df_merged['price'] / 1000)  # Convert MWh to kWh


df_lankstus = df_merged.groupby(['month', 'OBJ_NUMERIS'])['Lankstus_Hourly_Cost'].sum().reset_index()

# Add the fixed monthly fee
df_lankstus['Lankstus'] = 3.00 + df_lankstus['Lankstus_Hourly_Cost']


In [255]:
# Combine all plans into a single DataFrame
combined_plans = df_minimalus[['month', 'OBJ_NUMERIS', 'Minimalus']].merge(
    df_standartinis[['month', 'OBJ_NUMERIS', 'Standartinis']],
    on=['month', 'OBJ_NUMERIS']
).merge(
    df_lankstus[['month', 'OBJ_NUMERIS', 'Lankstus']],
    on=['month', 'OBJ_NUMERIS']
)

# Identify the cheapest plan for each apartment and month
combined_plans['Cheapest Plan'] = combined_plans[['Minimalus', 'Standartinis', 'Lankstus']].idxmin(axis=1)

combined_plans


,month,OBJ_NUMERIS,Minimalus,Standartinis,Lankstus,Cheapest Plan
0,2020-06,37082,51.11006,47.5546,30.101790,Lankstus
1,2020-06,37142,19.94222,19.2202,13.610828,Lankstus
2,2020-06,37502,6.31696,6.8336,5.881785,Lankstus
3,2020-06,40202,23.48928,22.4448,15.713195,Lankstus
4,2020-06,44282,12.06732,12.0612,8.872896,Lankstus
...,...,...,...,...,...,...
1699,2022-05,653246,29.42048,27.8368,34.920854,Standartinis
1700,2022-05,739422,60.65773,56.2343,72.873259,Standartinis
1701,2022-05,771822,21.89340,20.9940,27.052550,Standartinis
1702,2022-05,802542,90.47246,83.3386,113.804783,Standartinis


In [256]:
heatmap_data = combined_plans.pivot(index='OBJ_NUMERIS', columns='month', values='Cheapest Plan')

# Map plan names to numeric values for the heatmap
plan_mapping = {'Minimalus': 1, 'Standartinis': 2, 'Lankstus': 3}
heatmap_data = heatmap_data.replace(plan_mapping)

# Create the heatmap
fig = go.Figure(data=go.Heatmap(
    z=heatmap_data.values,  
    x=heatmap_data.columns.astype(str),  
    y=heatmap_data.index, 
    colorscale='Viridis',
    showscale=False,  
    xgap=2,
    ygap=2
    
))

custom_colors = {1: '#440154', 2: '#21908d', 3: '#fde725'}  
legend_items = {'Minimalus': 1, 'Standartinis': 2, 'Lankstus': 3}

# Add custom legend using scatter traces
for plan_name, value in legend_items.items():
    fig.add_trace(go.Scatter(
        x=[None], y=[None],
        mode='markers',
        marker=dict(size=10, color=custom_colors[value]),
        name=plan_name  
    ))


# Update layout
fig.update_layout(
    title='Cheapest Electricity Plan per Apartment per Month',
    xaxis_title='Month',
    yaxis_title='Apartment ID',
    xaxis=dict(type='category'),  
    yaxis=dict(type='category'),
    height=800,
    width=1000
)


fig.show()


C:\Users\baubl\AppData\Local\Temp\ipykernel_39008\2926729743.py:5: FutureWarning:

Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [261]:
# Count the number of times each plan is the cheapest for each OBJ_NUMERIS
plan_counts = combined_plans.groupby(['OBJ_NUMERIS', 'Cheapest Plan']).size().unstack(fill_value=0)

# Identify the apartment most frequently favoring "Minimalus" plan
minimalus_favored = plan_counts['Minimalus'].idxmax()
print(f"Apartment (OBJ_NUMERIS) most frequently favoring 'Minimalus' plan: {minimalus_favored}")

# Identify the apartment most frequently favoring "Lankstus" plan
lankstus_favored = plan_counts['Lankstus'].idxmax()
print(f"Apartment (OBJ_NUMERIS) most frequently favoring 'Lankstus' plan: {lankstus_favored}")

Apartment (OBJ_NUMERIS) most frequently favoring 'Minimalus' plan: 569298
Apartment (OBJ_NUMERIS) most frequently favoring 'Lankstus' plan: 567918


Apartment (OBJ_NUMERIS) most frequently favoring 'Minimalus' plan: 569298


Apartment (OBJ_NUMERIS) most frequently favoring 'Lankstus' plan: 567918